In [ ]:
!nvidia-smi

In [ ]:
!pip install libmr

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential,layers
from tensorflow.keras.layers import Conv3D,BatchNormalization, MaxPool2D, Activation, Flatten, Dense, GlobalAveragePooling3D, GlobalMaxPool3D, AveragePooling3D, Lambda, Reshape, UpSampling2D, Conv2DTranspose 
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
import cv2
import os
import numpy as np
import random
import scipy.io as sio
import numpy as np
import datetime
import sklearn
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from itertools import repeat
import matplotlib
import libmr

In [ ]:
# Data preparation

def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)  # X :(145, 145, 30) --> (195, 195, 30) with window =25
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))  # (21025, 25, 25, 30)   
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))  # (21025,)
    patchIndex = 0
    
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]  
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]            
            patchIndex = patchIndex + 1
  
    patchesData = np.expand_dims(patchesData, axis=-1)
    return patchesData,patchesLabels

def patches_class(X,Y,n):
    n_classes = n
    patches_list = []
    labeles_list = []
    for i in range(1,n_classes+1):   # not considering class 0
        patchesData_Ith_Label = X[Y==i,:,:,:,:]
        Ith_Label = Y[Y==i]
        patches_list.append(patchesData_Ith_Label)
        labeles_list.append(Ith_Label)
        
    return patches_list,labeles_list

In [ ]:
windowSize = 11
im_height, im_width, im_depth, im_channel = windowSize, windowSize, 30, 1 

In [ ]:
X = sio.loadmat('/content/drive/MyDrive/data/Indian_pines_corrected.mat')['indian_pines_corrected']
y = sio.loadmat('/content/drive/MyDrive/data/Indian_pines_gt.mat')['indian_pines_gt']
print(X.shape, y.shape)

X,pca = applyPCA(X,numComponents=im_depth)
print(X.shape, y.shape)

X, y = createImageCubes(X, y, windowSize)
print(X.shape, y.shape)

patches_class_ip,label_ip = patches_class(X,y,16) # class_wise list of patches #(16,) for class 0: (2009, 9, 9, 20, 1)

In [ ]:
# Assign label of coarse classses
# -------------------------------
# vegetation - 0
# Grasslands - 1
# woodland - 2
# Urban - 3
# water - 4

# Map True classes to coarse classes
# ----------------------------------
# True-class     coarse-class   coarse-ID
# - - - - - - - - - - - - - - - - - -  -  -
#       1	        vegetation        0
#       2	        vegetation        0  
#       3	        vegetation        0
#       4	        vegetation        0
#       5	        Grasslands        1
#       6	        Grasslands        1
#       7	        Grasslands        1
#       8	        Grasslands        1
#       9	        vegetation        0
#      10	        vegetation        0
#      11	        vegetation        0
#      12	        vegetation        0
#      13	        vegetation        0
#      14	        woodland          2
#      15	          Urban           3
#      16	          Urban           3

y_coarse = dict()
y_coarse[1] = [1,0,0,0,0]    # ToDo : [1,0,0,0]
y_coarse[2] = [1,0,0,0,0]
y_coarse[3] = [1,0,0,0,0]
y_coarse[4] = [1,0,0,0,0]
y_coarse[5] = [0,1,0,0,0]
y_coarse[6] = [0,1,0,0,0]
y_coarse[7] = [0,1,0,0,0]
y_coarse[8] = [0,1,0,0,0]
y_coarse[9] = [1,0,0,0,0]
y_coarse[10] = [1,0,0,0,0]
y_coarse[11] = [1,0,0,0,0]
y_coarse[12] = [1,0,0,0,0]
y_coarse[13] = [1,0,0,0,0]
y_coarse[14] = [0,0,1,0,0]
y_coarse[15] = [0,0,0,1,0]
y_coarse[16] = [0,0,0,1,0]


In [ ]:
coarse_dict = dict()
coarse_dict[1] = 0    # ToDo : [1,0,0,0]
coarse_dict[2] = 0
coarse_dict[3] = 0
coarse_dict[4] = 0
coarse_dict[5] = 1
coarse_dict[6] = 1
coarse_dict[7] = 1
coarse_dict[8] = 1
coarse_dict[9] = 0
coarse_dict[10] = 0
coarse_dict[11] = 0
coarse_dict[12] = 0
coarse_dict[13] = 0
coarse_dict[14] = 2
coarse_dict[15] = 3
coarse_dict[16] = 3

In [ ]:
train_class_indices = [1,2,4,5,7,9,10,11,13,14]    # 10 classes  
train_class_labels = [2,3,5,6,8,10,11,12,14,15]

test_class_indices = [0,3,6,8,12,15]               # 6 classes
test_class_labels = [1,4,7,9,13,16]


In [ ]:
# Definition of Episodes

# replace=False - no repeat
def new_episode(patches_list,NS,NQ,CS,CQ,class_labels) :  # NS 5,NQ 15,CS 3,CQ 6
    selected_classes = list(np.random.choice(class_labels,CQ,replace=False))  # Randomly choice 6 Query Classes
    support_classes = list(np.random.choice(selected_classes,CS,replace=False))  # Randomly choice 3 Support Classes from Q
    
    tquery_patches,tsupport_patches = [],[]
    query_labels,support_labels = [],[]
    
    for x in support_classes :      #3
        sran_indices = np.random.choice(patches_list[x-1].shape[0],NS,replace=False) # K=5 img nos from 20 per class for Support
        support_patches = patches_list[x-1][sran_indices,:,:,:,:]  # for x class those 5 nos
        tsupport_patches.extend(support_patches)               # 3 class * 5 patch per class = 15 patches
        for i in range(NS) :
            support_labels.append(x)                           # 3 class *5 nos = 15 nos
        
    for x in selected_classes :     #6
        qran_indices = np.random.choice(patches_list[x-1].shape[0],NQ,replace=False) # NQ=15 img nos from 20 per class for Query
        query_patches = patches_list[x-1][qran_indices,:,:,:,:]    # for x class those 15 nos
        tquery_patches.extend(query_patches)                   # 6 class * 15 patch per class = 90 patches
        for i in range(NQ) :
            query_labels.append(x)                             # 6 class * 15 nos = 90 nos
    
    temp1 = list(zip(tquery_patches, query_labels)) 
    random.shuffle(temp1)        # By Doing Shuffling, Support, Query Same class combination got mismatched - mitigated by support index
    tquery_patches, query_labels = zip(*temp1)
    
    tquery_patches = tf.convert_to_tensor(np.reshape(np.asarray(tquery_patches),(CQ*NQ,11,11,30,1)),dtype=tf.float32)
    tsupport_patches = tf.convert_to_tensor(np.reshape(np.asarray(tsupport_patches),(CS*NS,11,11,30,1)),dtype=tf.float32)
    return tquery_patches, tsupport_patches, query_labels, support_labels, support_classes    

In [ ]:
# Coarse Episode definition

def new_episode_coarse(patches_list,NQ,CS,CQ,Total_labels,S_Proto_classes) :  # NS 5,NQ 15,CS 3,CQ 6   
    U_c = list(np.random.choice( list(set(Total_labels).difference(set(S_Proto_classes))),CQ-CS,replace=False)) #train_class_labels
    query_classes = S_Proto_classes.copy()
    query_classes.extend(U_c)
    
    tquery_patches = []
    query_labels = []

    for x in query_classes :     #6
        qran_indices = np.random.choice(patches_list[x-1].shape[0],NQ,replace=False) # NQ=15 img nos from 20 per class for Query
        query_patches = patches_list[x-1][qran_indices,:,:,:,:]    # for x class those 15 nos
        tquery_patches.extend(query_patches)                   # 6 class * 15 patch per class = 90 patches
        for i in range(NQ) :
          if x==query_classes[0]:
            query_labels.append([1,0,0,0,0])                             # 6 class * 15 nos = 90 nos
          elif x==query_classes[1]:
            query_labels.append([0,1,0,0,0]) 
          elif x==query_classes[2]:
            query_labels.append([0,0,1,0,0]) 
          elif x==query_classes[3]:
            query_labels.append([0,0,0,1,0])  
          else:
            query_labels.append([0,0,0,0,1])             
 
    OOD_GT = []
    for l in query_classes:
      O_class = np.argmax(y_coarse[l]) # convert to correponding outlier class
      OOD_GT.extend(repeat(O_class,NQ))

    tquery_patches = tf.convert_to_tensor(np.reshape(np.asarray(tquery_patches),(CQ*NQ,11,11,30,1)),dtype=tf.float32)

    return tquery_patches, OOD_GT

In [ ]:
# Quadruplet Loss Definition

def calc_Quadruplet_dists(Proto, query, query_labels):  # [3,64], [90,64], [90,3]
    
    nbprototypes = Proto.shape[0]    # 3
    nbqueries = query.shape[0]       # 90
    Quadruplet_P_Nk, Quadruplet_P_Nu, Quadruplet_Nk_Nu = [], [], []
    Alpha1, Alpha2, Alpha3 = 0.5, 1, 0.5
    
    for i in range(nbprototypes):
        pos_ind = np.where(query_labels[:,i]==1)   # indexes of Positive queries for i th prototype
        pos_ind = list(pos_ind[0])            # indexes of Positive queries for i th prototype
        Neg_ind = list(set(np.arange(nbqueries)).difference(set(pos_ind)))  # indexes of Negative queries for i th prototype
        Neg_ind_U = []
        for u in range(nbqueries):
          if sum(query_labels[u])==0:
            Neg_ind_U.append(u)
        Neg_ind_K = list(set(Neg_ind).difference(set(Neg_ind_U)))

        Anchor = tf.expand_dims(Proto[i], 0)     # [1, 64]       # ith Prototype / Anchor
        
        Positive_d = []
        for j in range(len(pos_ind)):   # 15
            Pos = tf.expand_dims(query[pos_ind[j]], 0)   # [1, 64]
            Pos_dist = tf.reduce_mean(tf.math.pow(Anchor-Pos, 2), 1)   # scalar
            Positive_d.append(Pos_dist)                    # List
        Positive_d = tf.reduce_mean(Positive_d) # scalar
        
        Negative_d_k = []
        for j in range(len(Neg_ind_K)):
            Neg = tf.expand_dims(query[Neg_ind_K[j]], 0)   # [1, 64]
            Neg_dist = tf.reduce_mean(tf.math.pow(Anchor-Neg, 2), 1)   # scalar
            Negative_d_k.append(Neg_dist)                     # List
        Negative_d_k = tf.reduce_mean(Negative_d_k) # scalar

        Negative_d_U = []
        for j in range(len(Neg_ind_U)):
            Neg = tf.expand_dims(query[Neg_ind_U[j]], 0)   # [1, 64]
            Neg_dist = tf.reduce_mean(tf.math.pow(Anchor-Neg, 2), 1)   # scalar
            Negative_d_U.append(Neg_dist)                     # List
        Negative_d_U = tf.reduce_mean(Negative_d_U) # scalar
        
        P_Nk = Positive_d-Negative_d_k
        P_Nu = Positive_d-Negative_d_U
        Nk_Nu = Negative_d_k-Negative_d_U        
        Quadruplet_P_Nk.append(P_Nk)            #list
        Quadruplet_P_Nu.append(P_Nu) 
        Quadruplet_Nk_Nu.append(Nk_Nu) 
        
    P_Nk_Proto_mean = tf.reduce_mean(Quadruplet_P_Nk)    # p_dist - n_dist  # averaged for all Prototypes   # scalar 
    P_Nu_Proto_mean = tf.reduce_mean(Quadruplet_P_Nu)
    Nk_Nu_Proto_mean = tf.reduce_mean(Quadruplet_Nk_Nu)

    return tf.math.maximum(P_Nk_Proto_mean + Alpha1, 0)+tf.math.maximum(P_Nu_Proto_mean + Alpha2, 0)+tf.math.maximum(Nk_Nu_Proto_mean + Alpha3, 0)  # scalar

# Define Model
###### CBAM3D Layer

In [ ]:
class Channel_Attention_3D(tf.keras.layers.Layer) :
    def __init__(self,C,ratio) :
        super(Channel_Attention_3D,self).__init__()
        self.avg_pool = GlobalAveragePooling3D()
        self.max_pool = GlobalMaxPool3D()
        self.activation = Activation('sigmoid')
        self.fc1 = Dense(C/ratio, activation = 'relu')
        self.fc2 = Dense(C)    
    def call(self,x) :
        avg_out1 = self.avg_pool(x)
        avg_out2 = self.fc1(avg_out1)
        avg_out3 = self.fc2(avg_out2)
        max_out1 = self.max_pool(x)
        max_out2 = self.fc1(max_out1)
        max_out3 = self.fc2(max_out2)
        add_out = tf.math.add(max_out3,avg_out3)
        channel_att = self.activation(add_out)
        return channel_att 
    
class Spatial_Attention_3D(tf.keras.layers.Layer) :
    def __init__(self) :
        super(Spatial_Attention_3D,self).__init__()
        self.conv3d = Conv3D(1,(7,7,7),padding='same',activation='sigmoid')
        self.avg_pool_chl = Lambda(lambda x:tf.keras.backend.mean(x,axis=4,keepdims=True))
        self.max_pool_chl = Lambda(lambda x:tf.keras.backend.max(x,axis=4,keepdims=True)) 
    def call(self,x) :
        avg_out1 = self.avg_pool_chl(x)
        max_out1 = self.max_pool_chl(x)
        concat_out = tf.concat([avg_out1,max_out1],axis=-1)
        spatial_att = self.conv3d(concat_out)
        return spatial_att 
    
class CBAM_3D(tf.keras.layers.Layer) :
    def __init__(self,C,ratio) :
        super(CBAM_3D,self).__init__()
        self.C = C
        self.ratio = ratio
        self.channel_attention = Channel_Attention_3D(self.C,self.ratio)
        self.spatial_attention = Spatial_Attention_3D()
    def call(self,y,H,W,D,C) :
        ch_out1 = self.channel_attention(y)
        ch_out2 = tf.expand_dims(ch_out1, axis=1)
        ch_out3 = tf.expand_dims(ch_out2, axis=2)
        ch_out4 = tf.expand_dims(ch_out3, axis=3)
        ch_out5 = tf.tile(ch_out4, multiples=[1,H,W,D,1])
        ch_out5 = tf.math.multiply(ch_out5,y)
        sp_out1 = self.spatial_attention(ch_out5)
        sp_out2 = tf.tile(sp_out1, multiples = [1,1,1,1,C])
        sp_out3 = tf.math.multiply(sp_out2,ch_out5)
        return sp_out3 

### R3CBAM

In [ ]:
input_layer = layers.Input(shape = (im_height, im_width, im_depth, im_channel))
out1 = layers.Conv3D(filters=8, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(input_layer)
out2 = CBAM_3D(out1.shape[4],4)(out1,out1.shape[1],out1.shape[2],out1.shape[3],out1.shape[4])
out2 = layers.Conv3D(filters=8, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out1)
out2 = CBAM_3D(out2.shape[4],4)(out2,out2.shape[1],out2.shape[2],out2.shape[3],out2.shape[4])
out3 = layers.Conv3D(filters=8, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out2)
out4 = layers.Add()([out1, out3])  #Concatenate()
out5 = layers.MaxPool3D(pool_size=(2, 2, 4), strides=None, padding='same')(out4)

out6 = layers.Conv3D(filters=16, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out5)
out6 = CBAM_3D(out6.shape[4],4)(out6,out6.shape[1],out6.shape[2],out6.shape[3],out6.shape[4])
out7 = layers.Conv3D(filters=16, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out6)
out7 = CBAM_3D(out7.shape[4],4)(out7,out7.shape[1],out7.shape[2],out7.shape[3],out7.shape[4])
out8 = layers.Conv3D(filters=16, kernel_size=(3,3,3), padding='same',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out7)
out9 = layers.Add()([out6, out8])  #Concatenate()
out10 = layers.MaxPool3D(pool_size=(2, 2, 2), strides=None, padding='same')(out9)
out10 = CBAM_3D(out10.shape[4],4)(out10,out10.shape[1],out10.shape[2],out10.shape[3],out10.shape[4])
out11 = layers.Conv3D(filters=32, kernel_size=(3,3,3), padding='valid',activation='relu',input_shape=(im_height, im_width, im_depth, im_channel))(out10)
out12 = layers.Flatten()(out11)
output_layer = layers.Dense(3, activation='relu')(out12)
FE_model = Model(inputs=input_layer,outputs=output_layer,name='R3CBAM')
FE_model.summary()

Model: "R3CBAM"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 11, 11, 30,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 11, 11, 30,   224         ['input_2[0][0]']                
                                8)                                                                
                                                                                                  
 conv3d_2 (Conv3D)              (None, 11, 11, 30,   1736        ['conv3d[0][0]']                 
                                8)                                                           

### Outlier coarse classifier

In [ ]:
# Outlier Coarse Classifier
OOD_input = layers.Input(shape = (3))
OOD1 = layers.Dense(3, activation='relu')(OOD_input)
OOD2 = layers.Dense(6, activation='relu')(OOD1)
OOD3 = layers.Dense(3, activation='relu')(OOD2)
OOD4 = layers.Dense(5, activation='softmax')(OOD3)
OOD_Classifier = Model(inputs=OOD_input,outputs=OOD4,name='OOD_Classifier')
OOD_Classifier.summary()


# Weibull fitting and Calibration by P-OpenMax layer

In [ ]:
# Weibull fitting definition

# WEIBULL FIT
def calc_supportL2_dists(x, y):
  # x : (d,)  # mean
  # y : (m,d)  # features of same class
  dist = []
  for s in y:
    dist.append(tf.reduce_mean(tf.math.pow(x - s, 2)).numpy())
  return dist

def weibull_tailfitting_Low(distance, tailsize=5):  # inside episode - finegrained learning- penalizes more
    mr = libmr.MR()   
    tailtofit = distance[:tailsize]                              
    # tailtofit = sorted(distance)[:tailsize]     
    mr.fit_low(tailtofit, len(tailtofit))       
    return [mr]

def weibull_tailfitting_High(distance, tailsize=5):   # epoch wise - preserves true distribution structure
    mr = libmr.MR()                              # learning from samples which are very distant known samples...covers distribution density 
    tailtofit = distance[:tailsize]
    # tailtofit = sorted(distance)[-tailsize:]     # last tailsize distance //  sorted(distance) --> small to big
    mr.fit_high(tailtofit, len(tailtofit))      
    return [mr]

def Build_weibull_modify(z_prototypes,z_proto,class_labels,mode):
  Prototypes, S_distances, S_weibull= [], [], [[libmr.MR()] for i in range(16)]
  p = 0
  #weibull_m = {}
  for Proto_categogical in z_prototypes: 
    supp_categogical = z_proto[p]   # support set features of each class
    S_dis = calc_supportL2_dists(Proto_categogical,supp_categogical)              # Pending: Each epsisode updat distance with prev distance difference?
    Prototypes.append(Proto_categogical)
    S_distances.append(S_dis)
    if mode=='episode':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_Low(S_dis)         
    elif mode == 'epoch':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_High(S_dis)
    p = p+1
    
  weibull_model = {}
  weibull_model['mean_proto'] = np.array(Prototypes)  # (3, 64)
  weibull_model['dist_proto'] = np.array(S_distances) # (3, 5) 
  weibull_model['model'] = S_weibull
  return weibull_model  


def Build_weibull(z_prototypes,z_proto,mode):
  Prototypes, S_distances, S_weibull= [], [], []
  p = 0
  #weibull_m = {}
  for Proto_categogical in z_prototypes: 
    supp_categogical = z_proto[p]   # support set features of each class
    S_dis = calc_supportL2_dists(Proto_categogical,supp_categogical)              # Pending: Each epsisode updat distance with prev distance difference?
    Prototypes.append(Proto_categogical)
    S_distances.append(S_dis)
    if mode=='episode':
      S_weibull.append(weibull_tailfitting_Low(S_dis))         
    elif mode == 'epoch':
      S_weibull.append(weibull_tailfitting_High(S_dis))
    p = p+1
    
  weibull_model = {}
  weibull_model['mean_proto'] = np.array(Prototypes)  # (3, 64)
  weibull_model['dist_proto'] = np.array(S_distances) # (3, 5) 
  weibull_model['model'] = S_weibull
  return weibull_model

def Calculate_OpenMax(qembed, predictions, weibull_model, NCLASSES):
  alpharank = NCLASSES
  Open_predictions = []
  
  for q in range(qembed.shape[0]): 
    Q_embed = qembed[q]                                                         
    prediction = predictions[q].numpy()                                          
    AV = qembed[q]                                                               
  
    ranked_list = prediction.argsort().ravel()[::-1]                            # largest to smallest probability index    
    alpha_weights = [((alpharank+1) - i)/float(alpharank) for i in range(1, alpharank+1)]            # [1.0, 0.66, 0.33]    if alpharank = 3
    ranked_alpha = np.zeros(NCLASSES)
    for i in range(len(alpha_weights)):
        ranked_alpha[ranked_list[i]] = alpha_weights[i]                         # Assign weightage at top alpharank pos of ranked_list> 1st pos of ranked_list higest weight # [0.66, 0.33, 1]

    openmax_Known, openmax_Unknown = [], []
    p=0
    for categoryid in range(NCLASSES):
      Q_distance = tf.reduce_mean(tf.math.pow(weibull_model['mean_proto'][p] - Q_embed, 2)).numpy()     
      wscore = weibull_model['model'][p][0].w_score(Q_distance)                                         
      
      modified_fc8_score = AV[categoryid] * (1 - wscore*ranked_alpha[categoryid])                       
      openmax_Known += [modified_fc8_score]                                                             
      openmax_Unknown += [AV[categoryid] - modified_fc8_score]                                          
      p=p+1

    prob_scores, prob_unknowns,k_scores = [], [], []
    for category in range(NCLASSES):
      k_scores += [np.exp(openmax_Known[category])]                                                     
    total_denominator = np.sum(np.exp(openmax_Known[:])) + np.exp(np.sum(openmax_Unknown[:]))          
    prob_scores += [k_scores / total_denominator]                                                       
    prob_unknowns += [np.exp(np.sum(openmax_Unknown[:]))/total_denominator]                             
    
    scores = np.mean(np.asarray(prob_scores), axis=0)                                                   
    unknowns = np.mean(np.asarray(prob_unknowns), axis=0)                                               
    openmax_scores = scores.tolist() + [unknowns]                                                       
    assert len(openmax_scores) == NCLASSES + 1

    Open_predictions.append(openmax_scores)                                                            
  
  Open_predictions = np.array(Open_predictions)   
  return Open_predictions

In [ ]:
# Weibull for Coarse Classes Definition

def Build_weibull_coarse(z_prototypes,z_proto,class_labels,mode):
  Prototypes, S_distances, S_weibull= [], [], [[libmr.MR()] for i in range(4)]
  p = 0
  #weibull_m = {}
  for Proto_categogical in z_prototypes: 
    supp_categogical = z_proto[p]   # support set features of each class
    S_dis = calc_supportL2_dists(Proto_categogical,supp_categogical)              # Pending: Each epsisode updat distance with prev distance difference?
    Prototypes.append(Proto_categogical)
    S_distances.append(S_dis)
    if mode=='episode':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_Low(S_dis)         
    elif mode == 'epoch':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_High(S_dis) 
    p = p+1
    
  weibull_model = {}
  weibull_model['mean_proto'] = np.array(Prototypes)  # (3, 64)
  weibull_model['dist_proto'] = np.array(S_distances) # (3, 5) 
  weibull_model['model'] = S_weibull
  return weibull_model  


def Build_weibull_modify(z_prototypes,z_proto,class_labels,mode):
  Prototypes, S_distances, S_weibull= [], [], [[libmr.MR()] for i in range(16)]
  p = 0
  #weibull_m = {}
  for Proto_categogical in z_prototypes: 
    supp_categogical = z_proto[p]   # support set features of each class
    S_dis = calc_supportL2_dists(Proto_categogical,supp_categogical)              # Pending: Each epsisode updat distance with prev distance difference?
    Prototypes.append(Proto_categogical)
    S_distances.append(S_dis)
    if mode=='episode':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_Low(S_dis)         
    elif mode == 'epoch':
      S_weibull[class_labels[p]-1]=weibull_tailfitting_High(S_dis)      
    p = p+1
    
  weibull_model = {}
  weibull_model['mean_proto'] = np.array(Prototypes)  # (3, 64)
  weibull_model['dist_proto'] = np.array(S_distances) # (3, 5) 
  weibull_model['model'] = S_weibull
  return weibull_model    

In [ ]:
# P-OpenMax Layer

def Calculate_OpenMax_modify(qembed, predictions, weibull_model, NCLASSES,class_labels):
  alpharank = NCLASSES
  Open_predictions = []
  
  for q in range(qembed.shape[0]): 
    Q_embed = qembed[q]                                                         
    prediction = predictions[q].numpy()                                         
    AV = qembed[q]                                                               
  
    ranked_list = prediction.argsort().ravel()[::-1]                            # largest to smallest probability index     # i.e. [2,0,1]
    alpha_weights = [((alpharank+1) - i)/float(alpharank) for i in range(1, alpharank+1)]            # [1.0, 0.66, 0.33]    if alpharank = 3
    ranked_alpha = np.zeros(NCLASSES)
    for i in range(len(alpha_weights)):
        ranked_alpha[ranked_list[i]] = alpha_weights[i]                         # Assign weightage at top alpharank pos of ranked_list> 1st pos of ranked_list higest weight # [0.66, 0.33, 1]

    openmax_Known, openmax_Unknown = [], []
    p=0
    for categoryid in range(NCLASSES):
      Q_distance = tf.reduce_mean(tf.math.pow(weibull_model['mean_proto'][p] - Q_embed, 2)).numpy()     
      wscore = weibull_model['model'][class_labels[p]-1][0].w_score(Q_distance)                                         
      
      modified_fc8_score = AV[categoryid] * (1 - wscore*ranked_alpha[categoryid])                       
      openmax_Known += [modified_fc8_score]                                                             
      openmax_Unknown += [AV[categoryid] - modified_fc8_score]                                          
      p=p+1

    prob_scores, prob_unknowns,k_scores = [], [], []
    for category in range(NCLASSES):
      k_scores += [np.exp(openmax_Known[category])]                                                     
    total_denominator = np.sum(np.exp(openmax_Known[:])) + np.exp(np.sum(openmax_Unknown[:]))           
    prob_scores += [k_scores / total_denominator]                                                       
    prob_unknowns += [np.exp(np.sum(openmax_Unknown[:]))/total_denominator]                            
    
    scores = np.mean(np.asarray(prob_scores), axis=0)                                                   
    unknowns = np.mean(np.asarray(prob_unknowns), axis=0)                                               
    openmax_scores = scores.tolist() + [unknowns]                                                       
    assert len(openmax_scores) == NCLASSES + 1

    Open_predictions.append(openmax_scores)                                                             
  
  Open_predictions = np.array(Open_predictions)   
  return Open_predictions

### Compute distance from Prototypes

In [ ]:
def calc_euclidian_dists(x, y):
  # x : (n,d)
  # y : (m,d)
    n = x.shape[0]
    m = y.shape[0]
    x = tf.tile(tf.expand_dims(x, 1), [1, m, 1])
    y = tf.tile(tf.expand_dims(y, 0), [n, 1, 1])
    return tf.reduce_mean(tf.math.pow(x - y, 2), 2)

In [ ]:
CQ = 6 #C2
CS = 3 #C1
N = 15
K = 5
optim = tf.keras.optimizers.Adam(0.001)    # FE
cce = tf.keras.losses.CategoricalCrossentropy()
OODoptim = tf.keras.optimizers.Adam(0.001)   # OOD

# Train

In [ ]:
%mkdir TIP_fsosr
checkpoint_dir = '/content/Fsosr_EVML'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optim,FE_model = FE_model, OOD_Classifier=OOD_Classifier)

In [ ]:
emb_dim = 3

def proto_train(ep_class_images,ep_query_images,ep_class_labels,ep_query_labels,support_classes,K,CS,CQ,N):#5,3,6,15     
    outlier = 0
    sembed = FE_model(ep_class_images)                             # [15, 3]        
    qembed = FE_model(ep_query_images)                             # [90, 3]
    y_query = np.asarray(np.zeros((len(ep_query_images),CS)),dtype=np.float32)  # (90, 3) 
    y_true = np.zeros(len(ep_query_labels)) #for storing labels of classes, 0 for unseen; 1,2,3 for the three classes    #(90)
    for i in range(len(ep_query_labels)) :
      if ep_query_labels[i] in support_classes :
            x = support_classes.index(ep_query_labels[i])
            y_query[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
            y_true[i] = x+1
    y_support = np.asarray(np.zeros((len(ep_class_images),CS)),dtype=np.float32)  # (15, 3) 
    for i in range(len(ep_class_labels)) :
      if ep_class_labels[i] in support_classes :
            x = support_classes.index(ep_class_labels[i])
            y_support[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
    z_proto = tf.reshape(sembed,[CS, K, sembed.shape[-1]])           # [3, 5, 3]    
    z_prototypes = tf.math.reduce_mean(z_proto, axis=1)              # [3, 3]   

    # Calculate closed class loss
    dists = calc_euclidian_dists(qembed, z_prototypes)  #sqembedK
    log_p_y = tf.nn.log_softmax(-dists,axis=-1)
    cec_closed_loss = -tf.reduce_mean((tf.reduce_sum(tf.multiply(y_query, log_p_y), axis=-1)))  #y_sqK     


    loss_quad=calc_Quadruplet_dists(z_prototypes, qembed, y_query)

    #  Calculate closed class accuracy 
    predictions_closed = tf.nn.softmax(-dists, axis=-1)
    pred_index = tf.argmax(predictions_closed,axis=-1)
    correct_pred = 0
    for i in range(len(ep_query_labels)) :
        if ep_query_labels[i] in support_classes:     #support_labels :
            x = support_classes.index(ep_query_labels[i])
            if x == pred_index[i] :
                correct_pred += 1          
    closed_oa = correct_pred/(CS*N)                     #(CQ*N)  # scalar

    # Calculate Open class loss
    NCLASSES = CS
    weibull_model = Build_weibull_modify(z_prototypes,z_proto,support_classes,'episode')    
    predictions_Open = Calculate_OpenMax_modify(qembed, predictions_closed, weibull_model, NCLASSES, support_classes) 

    y_q_open = np.concatenate((y_query,np.zeros((y_query.shape[0],1))),axis=1)  
    index = 0
    for ind in y_q_open:
      if np.sum(ind)==0:
        y_q_open[index, NCLASSES] = 1.
      index += 1
    cec_open_loss = tf.math.abs(-tf.reduce_mean((tf.reduce_sum(tf.multiply(y_q_open, predictions_Open), axis=-1))))
    cec_open_loss = tf.cast(cec_open_loss, dtype=tf.float32)
    
    # Calculate Open class accuracy
    pred_open_index = tf.argmax(predictions_Open,axis=-1)
    correct_open_index = tf.argmax(y_q_open,axis=-1)


    cm = confusion_matrix(correct_open_index,pred_open_index)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    open_oa = (sum(TP)+sum(TN))/(sum(TP)+sum(TN)+sum(FP)+sum(FN))

    # calculate Outlier detection accuracy 
    correct_outlier_index = (correct_open_index==NCLASSES)
    pred_outlier_index = (pred_open_index==NCLASSES)
    out_equality = tf.math.equal(pred_outlier_index, correct_outlier_index)
    outlier_det_acc = tf.math.reduce_mean(tf.cast(out_equality, tf.float32))
    outlier_det_acc = outlier_det_acc.numpy()

    # Outlier coarse classification
    Unkown_classes = list(set(ep_query_labels).difference(set(ep_class_labels)))
    U_query, U_gt, ind = [], [], 0
    for l in ep_query_labels:
      if l in Unkown_classes:
        U_query.append(qembed[ind])
        U_gt.append(y_coarse[l])
      ind = ind+1    
    U_query = tf.convert_to_tensor(np.array(U_query))
    U_gt = tf.convert_to_tensor(np.array(U_gt))    
    for i in range(1):
      with tf.GradientTape() as outlier_tape:
        Upred = OOD_Classifier(U_query) 
        outlier_loss = cce(U_gt, Upred)
      Ograds = outlier_tape.gradient(outlier_loss, OOD_Classifier.trainable_variables)
      OODoptim.apply_gradients(zip(Ograds, OOD_Classifier.trainable_variables))

    # coarseoa calculation
    u_gt=np.argmax(np.array(U_gt),1)
    u_pred=np.argmax(np.array(Upred),1)  
    coarse_oa=tf.math.reduce_mean(tf.cast(tf.math.equal(tf.convert_to_tensor(u_gt),tf.convert_to_tensor(u_pred)), tf.float32))
    coarse_oa=coarse_oa.numpy()
    # Multitask loss  
    loss = cec_open_loss + loss_quad + cec_closed_loss + outlier_loss

    return loss, closed_oa, outlier_det_acc, open_oa, coarse_oa, z_proto      # return z_proto (class-specific support vector)

# Metrics to gather
train_loss = tf.metrics.Mean(name='train_loss')
train_closedoa = tf.metrics.Mean(name='train_closedoa')
train_outlier_acc = tf.metrics.Mean(name='train_outlier_acc')
train_openoa = tf.metrics.Mean(name='train_openoa')
train_coarse_oa = tf.metrics.Mean(name='train_coarse_oa')

def train_step(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,K,CS,CQ,N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, accuracy, outlier_det_acc, openoa, coarse_oa, z_proto = proto_train(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,K,CS,CQ,N)
    gradients = tape.gradient(loss, FE_model.trainable_variables)
    optim.apply_gradients(zip(gradients, FE_model.trainable_variables))
    train_loss(loss)
    train_closedoa(accuracy)   
    train_outlier_acc(outlier_det_acc)
    train_openoa(openoa)
    train_coarse_oa(coarse_oa)
    return z_proto

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
#%load_ext tensorboard
#%tensorboard --logdir logs/gradient_tape    

OCATrain = []
for epoch in range(1000):                                       #ToDo
    train_loss.reset_states()  
    train_closedoa.reset_states()   
    train_outlier_acc.reset_states()
    train_openoa.reset_states()
    train_coarse_oa.reset_states()

    Sembed_Dict = {2:[], 3:[], 5:[], 6:[], 8:[], 10:[], 11:[], 12:[], 14:[], 15:[]}
    # [2,3,5,6,8,10,11,12,14,15]
    for epi in range(10): 
        tquery_patches, tsupport_patches, query_labels, support_labels, support_classes = new_episode(patches_class_ip,K,N,CS,CQ,train_class_labels)   
        z_p = train_step(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,K,CS,CQ,N)   # z_p = z_proto   #[3, 5, 3]
        #Accumulator: store return z_proto per support_classes per episode
        ind = 0
        for sl in support_classes:
          Sembed_Dict[sl].extend(np.array(z_p[ind]))
          ind +=1

        
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('closedoa', train_closedoa.result(), step=epoch)
        tf.summary.scalar('outlier_det_acc',train_outlier_acc.result(), step=epoch)
        tf.summary.scalar('openoa',train_openoa.result(), step=epoch)

    template = 'Epoch {}, Train Loss: {:.2f}, closed OA: {:.2f}, Open OA: {:.2f}, Outlier_Det. Acc: {:.2f}, Outlier course Acc: {:.2f}'
    print(template.format(epoch+1,train_loss.result(),train_closedoa.result()*100,train_openoa.result()*100,train_outlier_acc.result()*100,train_coarse_oa.result()*100))
    OCATrain.append(train_coarse_oa.result())
 
    checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/gradient_tape

# Fine Tuning

In [ ]:

train_patches_class = [patches_class_ip[i] for i in train_class_indices]        #(10)
test_patches_class = [patches_class_ip[i] for i in test_class_indices]        #(6) 
test_support_labels = [1,4,7]
ft_labels = [1,2,3,4,5,6,7,8,10,11,12,14,15]

In [ ]:
tune_set_5 = [[] for i in range(16)]
for j in range(1,17) :
  if j in train_class_labels :
    tune_set_5[j-1] = patches_class_ip[j-1] 
  elif j in test_support_labels :
    tune_set_5[j-1] = patches_class_ip[j-1][:5,:,:,:,:]

In [ ]:
print(len(train_patches_class))
print(train_patches_class[1].shape)
print(len(test_patches_class))
print(train_class_labels)
print(test_class_labels)

In [ ]:
def tune_episode(patches_list,NS,NQ,CS,CQ,class_labels) :  # NS 5,NQ 15,CS 3,CQ 6
    selected_classes = list(np.random.choice(class_labels,CQ,replace=False))  # Randomly choice 6 Query Classes
    support_classes = list(np.random.choice(selected_classes,CS,replace=False))  # Randomly choice 3 Support Classes from Q
    
    tquery_patches,tsupport_patches = [],[]
    query_labels,support_labels = [],[]
    
    for x in support_classes :      #3
        sran_indices = np.random.choice(patches_list[x-1].shape[0],NS,replace=False) # K=5 img nos from 20 per class for Support
        support_patches = patches_list[x-1][sran_indices,:,:,:,:]  # for x class those 5 nos
        tsupport_patches.extend(support_patches)               # 3 class * 5 patch per class = 15 patches
        for i in range(NS) :
            support_labels.append(x)                           # 3 class *5 nos = 15 nos
        
    for x in selected_classes :     #6
        qran_indices = np.random.choice(patches_list[x-1].shape[0],NQ,replace=False) # NQ=15 img nos from 20 per class for Query
        query_patches = patches_list[x-1][qran_indices,:,:,:,:]    # for x class those 15 nos
        tquery_patches.extend(query_patches)                   # 6 class * 15 patch per class = 90 patches
        for i in range(NQ) :
            query_labels.append(x)                             # 6 class * 15 nos = 90 nos
    
    temp1 = list(zip(tquery_patches, query_labels)) 
    random.shuffle(temp1)        # By Doing Shuffling, Support, Query Same class combination got mismatched - mitigated by support index
    tquery_patches, query_labels = zip(*temp1)
    
    tquery_patches = tf.convert_to_tensor(np.reshape(np.asarray(tquery_patches),(CQ*NQ,11,11,30,1)),dtype=tf.float32)
    tsupport_patches = tf.convert_to_tensor(np.reshape(np.asarray(tsupport_patches),(CS*NS,11,11,30,1)),dtype=tf.float32)
    return tquery_patches, tsupport_patches, query_labels, support_labels, support_classes    

In [ ]:
%mkdir Tune_TIP
checkpoint_dir_tune = '/content/Fsosr_EVML_Tune'         # To change
checkpoint_prefix_tune = os.path.join(checkpoint_dir_tune, "ckpt")
checkpoint_tune = tf.train.Checkpoint(optimizer=optim,FE_model = FE_model, OOD_Classifier=OOD_Classifier)

In [ ]:
emb_dim = 3
tK = 1
tN = 4

def proto_tune(ep_class_images,ep_query_images,ep_class_labels,ep_query_labels,support_classes,tK,CS,CQ,tN):   
    outlier = 0
    sembed = FE_model(ep_class_images)                             # [15, 3]        
    qembed = FE_model(ep_query_images)                             # [90, 3]
    y_query = np.asarray(np.zeros((len(ep_query_images),CS)),dtype=np.float32)  # (90, 3) 
    y_true = np.zeros(len(ep_query_labels)) #for storing labels of classes, 0 for unseen; 1,2,3 for the three classes    #(90)
    for i in range(len(ep_query_labels)) :
      if ep_query_labels[i] in support_classes :
            x = support_classes.index(ep_query_labels[i])
            y_query[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
            y_true[i] = x+1
    y_support = np.asarray(np.zeros((len(ep_class_images),CS)),dtype=np.float32)  # (15, 3) 
    for i in range(len(ep_class_labels)) :
      if ep_class_labels[i] in support_classes :
            x = support_classes.index(ep_class_labels[i])
            y_support[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
    z_proto = tf.reshape(sembed,[CS, tK, sembed.shape[-1]])           # [3, 5, 3]    
    z_prototypes = tf.math.reduce_mean(z_proto, axis=1)              # [3, 3]   

    # Calculate closed class loss
    dists = calc_euclidian_dists(qembed, z_prototypes)  
    log_p_y = tf.nn.log_softmax(-dists,axis=-1)
    cec_closed_loss = -tf.reduce_mean((tf.reduce_sum(tf.multiply(y_query, log_p_y), axis=-1)))  

    loss_quad=calc_Quadruplet_dists(z_prototypes, qembed, y_query)

    #  Calculate closed class accuracy 
    predictions_closed = tf.nn.softmax(-dists, axis=-1)
    pred_index = tf.argmax(predictions_closed,axis=-1)
    correct_pred = 0
    for i in range(len(ep_query_labels)) :
        if ep_query_labels[i] in support_classes:     #support_labels :
            x = support_classes.index(ep_query_labels[i])
            if x == pred_index[i] :
                correct_pred += 1          
    closed_oa = correct_pred/(CS*tN)                     #(CQ*N)  # scalar

    # Calculate Open class loss
    NCLASSES = CS
    weibull_model = Build_weibull_modify(z_prototypes,z_proto,support_classes,'episode')    
    predictions_Open = Calculate_OpenMax_modify(qembed, predictions_closed, weibull_model, NCLASSES, support_classes) 

    y_q_open = np.concatenate((y_query,np.zeros((y_query.shape[0],1))),axis=1)   
    index = 0
    for ind in y_q_open:
      if np.sum(ind)==0:
        y_q_open[index, NCLASSES] = 1.
      index += 1
    cec_open_loss = tf.math.abs(-tf.reduce_mean((tf.reduce_sum(tf.multiply(y_q_open, predictions_Open), axis=-1))))
    cec_open_loss = tf.cast(cec_open_loss, dtype=tf.float32)
    
    # Calculate Open class accuracy
    pred_open_index = tf.argmax(predictions_Open,axis=-1)
    correct_open_index = tf.argmax(y_q_open,axis=-1)

    cm = confusion_matrix(correct_open_index,pred_open_index)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    open_oa = (sum(TP)+sum(TN))/(sum(TP)+sum(TN)+sum(FP)+sum(FN))

    # calculate Outlier detection accuracy 
    correct_outlier_index = (correct_open_index==NCLASSES)
    pred_outlier_index = (pred_open_index==NCLASSES)
    out_equality = tf.math.equal(pred_outlier_index, correct_outlier_index)
    outlier_det_acc = tf.math.reduce_mean(tf.cast(out_equality, tf.float32))
    outlier_det_acc = outlier_det_acc.numpy()

    # Outlier coarse classification
    Unkown_classes = list(set(ep_query_labels).difference(set(ep_class_labels)))
    U_query, U_gt, ind = [], [], 0
    for l in ep_query_labels:
      if l in Unkown_classes:
        U_query.append(qembed[ind])
        U_gt.append(y_coarse[l])
      ind = ind+1    
    U_query = tf.convert_to_tensor(np.array(U_query))
    U_gt = tf.convert_to_tensor(np.array(U_gt))    
    with tf.GradientTape() as outlier_tape:
      Upred = OOD_Classifier(U_query) 
      outlier_loss = cce(U_gt, Upred)
    Ograds = outlier_tape.gradient(outlier_loss, OOD_Classifier.trainable_variables)
    OODoptim.apply_gradients(zip(Ograds, OOD_Classifier.trainable_variables))

    # coarseoa calculation
    u_gt=np.argmax(np.array(U_gt),1)
    u_pred=np.argmax(np.array(Upred),1)  
    coarse_oa=tf.math.reduce_mean(tf.cast(tf.math.equal(tf.convert_to_tensor(u_gt),tf.convert_to_tensor(u_pred)), tf.float32))
    coarse_oa=coarse_oa.numpy()
    # Multitask loss  

    loss = cec_open_loss + loss_quad + cec_closed_loss + outlier_loss
    return loss, closed_oa, outlier_det_acc, open_oa, coarse_oa, z_proto      # return z_proto (class-specific support vector)

# Metrics to gather
tune_loss = tf.metrics.Mean(name='train_loss')
tune_closedoa = tf.metrics.Mean(name='train_closedoa')
tune_outlier_acc = tf.metrics.Mean(name='train_outlier_acc')
tune_openoa = tf.metrics.Mean(name='train_openoa')
tune_coarse_oa = tf.metrics.Mean(name='tune_coarse_oa')

def tune_step(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,tK,CS,CQ,tN):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, accuracy, outlier_det_acc, openoa, coarse_oa, z_proto = proto_train(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,tK,CS,CQ,tN)
    gradients = tape.gradient(loss, FE_model.trainable_variables)
    optim.apply_gradients(zip(gradients, FE_model.trainable_variables))
    tune_loss(loss)
    tune_closedoa(accuracy)   
    tune_outlier_acc(outlier_det_acc)
    tune_openoa(openoa)
    tune_coarse_oa(coarse_oa)
    return z_proto

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tune_log_dir = 'logs/gradient_tape/' + current_time + '/tune'
tune_summary_writer = tf.summary.create_file_writer(train_log_dir)
#%load_ext tensorboard
#%tensorboard --logdir logs/gradient_tape    

OCATune = []
for epoch in range(1000):                                       #ToDo
    tune_loss.reset_states()  
    tune_closedoa.reset_states()   
    tune_outlier_acc.reset_states()
    tune_openoa.reset_states()
    tune_coarse_oa.reset_states()

    Sembed_Dict = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[], 12:[], 10:[], 15:[], 16:[]}

    for epi in range(10): 
        tquery_patches, tsupport_patches, query_labels, support_labels, support_classes = tune_episode(tune_set_5,1,4,3,6,ft_labels)   
        z_p = tune_step(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,tK,CS,CQ,tN)   # z_p = z_proto   #[3, 5, 3]
        #Accumulator: store return z_proto per support_classes per episode

        
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', tune_loss.result(), step=epoch)
        tf.summary.scalar('closedoa', tune_closedoa.result(), step=epoch)
        tf.summary.scalar('outlier_det_acc',tune_outlier_acc.result(), step=epoch)
        tf.summary.scalar('openoa',tune_openoa.result(), step=epoch)

    template = 'Epoch {}, Train Loss: {:.2f}, closed OA: {:.2f}, Open OA: {:.2f}, Outlier_Det. Acc: {:.2f}, Outlier course Acc: {:.2f}'
    print(template.format(epoch+1,tune_loss.result(),tune_closedoa.result()*100,tune_openoa.result()*100,tune_outlier_acc.result()*100,tune_coarse_oa.result()*100))
    OCATune.append(tune_coarse_oa.result())
 
    checkpoint_tune.save(file_prefix = checkpoint_prefix_tune)

# Testing

In [ ]:

train_patches_class = [patches_class_ip[i] for i in train_class_indices]        #(10)
test_patches_class = [patches_class_ip[i] for i in test_class_indices]        #(6) 
test_support_labels = [1,4,7]
ft_labels = [1,2,3,4,5,6,7,8,10,11,12,14,15]

In [ ]:
def proto_test(ep_class_images,ep_query_images,ep_class_labels,ep_query_labels,support_classes,K,CS,CQ,N):#5,3,6,15     
    outlier = 0
    sembed = FE_model(ep_class_images)                             # [15, 64]        
    qembed = FE_model(ep_query_images)                             # [90, 64]
    y_query = np.asarray(np.zeros((len(ep_query_images),CS)),dtype=np.float32)  # (90, 3) 
    y_true = np.zeros(len(ep_query_labels)) #for storing labels of classes, 0 for unseen; 1,2,3 for the three classes
    y_auc = np.zeros((len(ep_query_labels))) #for storing labels, 1 for seen, and 0 for unseen
    for i in range(len(ep_query_labels)) :
      if ep_query_labels[i] in support_classes :
            x = support_classes.index(ep_query_labels[i])
            y_query[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
            y_true[i] = x+1
            y_auc[i] = 1
    y_support = np.asarray(np.zeros((len(ep_class_images),CS)),dtype=np.float32)  # (15, 3) 
    for i in range(len(ep_class_labels)) :
      if ep_class_labels[i] in support_classes :
            x = support_classes.index(ep_class_labels[i])
            y_support[i][x] = 1.                                      # [[0., 0., 1.], [0., 0., 0.], ... (90,3)
    z_proto = tf.reshape(sembed,[CS, K, sembed.shape[-1]])           # [3, 5, 64]
    z_prototypes = tf.math.reduce_mean(z_proto, axis=1)        # [3, 64]   
    
    
    # Calculate closed class loss
    dists = calc_euclidian_dists(qembed, z_prototypes) 
    log_p_y = tf.nn.log_softmax(-dists,axis=-1)
    cec_closed_loss = -tf.reduce_mean((tf.reduce_sum(tf.multiply(y_query, log_p_y), axis=-1))) 

    #  Calculate closed class accuracy 
    predictions_closed  = tf.nn.softmax(-dists, axis=-1)
    pred_index = tf.argmax(predictions_closed ,axis=-1)
    correct_pred = 0
    for i in range(len(ep_query_labels)) :
        if ep_query_labels[i] in support_classes:     #support_labels :
            x = support_classes.index(ep_query_labels[i])
            if x == pred_index[i] :
                correct_pred += 1          
    closed_oa = correct_pred/(CS*N)                     #(CQ*N)  # scalar

    # Calculate Open class loss
    NCLASSES = CS
    weibull_model = Build_weibull_modify(z_prototypes,z_proto,support_classes,'episode')               
    predictions_Open = Calculate_OpenMax_modify(qembed, predictions_closed, weibull_model, NCLASSES, support_classes)

    y_q_open = np.concatenate((y_query,np.zeros((y_query.shape[0],1))),axis=1)    
    index = 0
    for ind in y_q_open:
      if np.sum(ind)==0:
        y_q_open[index, NCLASSES] = 1.
      index += 1
    cec_open_loss = tf.math.abs(-tf.reduce_mean((tf.reduce_sum(tf.multiply(y_q_open, predictions_Open), axis=-1))))
    cec_open_loss = tf.cast(cec_open_loss, dtype=tf.float32)
    
    # Calculate Open class accuracy
    pred_open_index = tf.argmax(predictions_Open,axis=-1)
    correct_open_index = tf.argmax(y_q_open,axis=-1)

    cm = confusion_matrix(correct_open_index,pred_open_index)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    open_oa = (sum(TP)+sum(TN))/(sum(TP)+sum(TN)+sum(FP)+sum(FN))
    

    # calculate Outlier detection accuracy 
    correct_outlier_index = (correct_open_index==NCLASSES)
    pred_outlier_index = (pred_open_index==NCLASSES)
    out_equality = tf.math.equal(pred_outlier_index, correct_outlier_index)
    outlier_det_acc = tf.math.reduce_mean(tf.cast(out_equality, tf.float32))
    outlier_det_acc = outlier_det_acc.numpy()

    # outlier coarse accuracy
    coarse_Open_pred = []
    OOD_GT = []
    for q in range(len(predictions_Open)):
      coarse_Open_pred.append(np.argmax(predictions_Open[q]))  
    coarse_OOD_ind = [i for i, x in enumerate(coarse_Open_pred) if x==3]   # ouliers will be always at 3


    # compute AUROC
    y_score = np.zeros((len(ep_query_labels)))
    for i in range(len(ep_query_labels)) :
      y_score[i] = predictions_Open[i,3]    # outlier prob > openmax last class
    auc = sklearn.metrics.roc_auc_score(y_auc, y_score)
    
    
    Unkown_classes = list(set(ep_query_labels).difference(set(ep_class_labels)))
    U_query, U_gt, ind = [], [], 0
    for l in ep_query_labels:
      if l in Unkown_classes:
        U_query.append(qembed[ind])
        U_gt.append(y_coarse[l])
      ind = ind+1    
    U_query = tf.convert_to_tensor(np.array(U_query))
    U_gt = tf.convert_to_tensor(np.array(U_gt))    
    Upred = OOD_Classifier(U_query) 

    # coarseoa calculation
    u_gt=np.argmax(np.array(U_gt),1)
    u_pred=np.argmax(np.array(Upred),1)  
    coarse_oa=tf.math.reduce_mean(tf.cast(tf.math.equal(tf.convert_to_tensor(u_gt),tf.convert_to_tensor(u_pred)), tf.float32))
    coarse_oa=coarse_oa.numpy()

    # Multitask loss
    loss = cec_closed_loss +cec_open_loss
    #print(cec_closed_loss,cec_open_loss,loss)
    return loss, closed_oa, outlier_det_acc, open_oa, coarse_oa, auc      

In [ ]:

def test_episode(patches_list,NS,NQ,CS,CQ) :  # NS 5,NQ 15,CS 3,CQ 6
    selected_classes = test_class_labels  # 6 Query Classes
    support_classes = test_support_labels  # 3 Support Classes from Q
    
    tquery_patches,tsupport_patches = [],[]
    query_labels,support_labels = [],[]
    
    for x in support_classes :      #3
        sran_indices = np.random.choice(patches_list[x-1].shape[0],NS,replace=False) # K=5 img nos from 20 per class for Support
        support_patches = patches_list[x-1][sran_indices,:,:,:,:]  # for x class those 5 nos
        tsupport_patches.extend(support_patches)               # 3 class * 5 patch per class = 15 patches
        for i in range(NS) :
            support_labels.append(x)                           # 3 class *5 nos = 15 nos
        
    for x in selected_classes :     #6
        qran_indices = np.random.choice(patches_list[x-1].shape[0],NQ,replace=False) # NQ=15 img nos from 20 per class for Query
        query_patches = patches_list[x-1][qran_indices,:,:,:,:]    # for x class those 15 nos
        tquery_patches.extend(query_patches)                   # 6 class * 15 patch per class = 90 patches
        for i in range(NQ) :
            query_labels.append(x)                             # 6 class * 15 nos = 90 nos
    
    temp1 = list(zip(tquery_patches, query_labels)) 
    random.shuffle(temp1)        # By Doing Shuffling, Support, Query Same class combination got mismatched - mitigated by support index
    tquery_patches, query_labels = zip(*temp1)
    
    tquery_patches = tf.convert_to_tensor(np.reshape(np.asarray(tquery_patches),(CQ*NQ,11,11,30,1)),dtype=tf.float32)
    tsupport_patches = tf.convert_to_tensor(np.reshape(np.asarray(tsupport_patches),(CS*NS,11,11,30,1)),dtype=tf.float32)
    return tquery_patches, tsupport_patches, query_labels, support_labels, support_classes    

In [ ]:
total_acc = 0 
total_open_oa = 0
total_outlier_acc = 0 
total_outlier_Course_acc = 0 
total_auroc = 0

tepochs = 2
K = 5
N = 15
for i in range(tepochs) :
  tquery_patches, tsupport_patches, query_labels, support_labels, support_classes = test_episode(patches_class_ip,5,15,3,6)   
  loss, closed_oa, outlier_det_acc, open_oa, OOD_course_acc, auc = proto_test(tsupport_patches,tquery_patches,support_labels,query_labels,support_classes,5,3,6,15)
  total_acc = total_acc + closed_oa
  total_open_oa = total_open_oa + open_oa
  total_outlier_acc = total_outlier_acc + outlier_det_acc
  total_outlier_Course_acc = total_outlier_Course_acc + OOD_course_acc
  total_auroc = total_auroc + auc
print('closed accuracy',total_acc*100/tepochs)
print('open oa',total_open_oa*100/tepochs)
print('Outlier detection accuracy', (total_outlier_acc*100/tepochs))
print('Outlier coarse accuracy',total_outlier_Course_acc*100/tepochs)
print('AUROC',total_auroc*100/tepochs)